In [1]:
#!pip install opencv-python
#!pip install jupyter-xml
#!pip install Pillow
#!pip install pytesseract
#!pip install tesseract
#!pip install scikit-image
#!pip install pytest-shutil
#!pip install pandas
#!pip install plotly
#!pip install matplotlib
#!pip install scikit-learn
#!pip install torch
#!pip install onnx-simplifier

In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import pytesseract as pt
import plotly.express as px
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet

from glob import glob
from skimage import io
from shutil import copy
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

## Convert xml to csv

In [3]:
cols = ["filepath","xmin", "xmax", "ymin", "ymax"] 
rows = [] 
path = glob('C:/Users/vivek/OneDrive/Documents/Machine learning/Detect-and-recognized-car-license/annotations/*.xml')
  
# Parsing the XML file 
for filename in path: 
    xmlparse = xet.parse(filename) 
    root = xmlparse.getroot() 
    member_object = root.find('object')
    i = member_object.find('bndbox')
    xmin = i.find("xmin").text 
    xmax = i.find("xmax").text 
    ymin = i.find("ymin").text 
    ymax = i.find("ymax").text 
  
    rows.append({"filepath":filename,
                 "xmin": xmin, 
                 "xmax": xmax, 
                 "ymin": ymin, 
                 "ymax": ymax}) 
df = pd.DataFrame(rows, columns=cols) 
  
# Writing dataframe to csv 
df.to_csv('output.csv') 

In [4]:
df

,filepath,xmin,xmax,ymin,ymax
0,C:/Users/vivek/OneDrive/Documents/Machine lear...,226,419,125,173
1,C:/Users/vivek/OneDrive/Documents/Machine lear...,134,262,128,160
2,C:/Users/vivek/OneDrive/Documents/Machine lear...,140,303,5,148
3,C:/Users/vivek/OneDrive/Documents/Machine lear...,175,214,114,131
4,C:/Users/vivek/OneDrive/Documents/Machine lear...,167,240,202,220
...,...,...,...,...,...
428,C:/Users/vivek/OneDrive/Documents/Machine lear...,278,407,182,263
429,C:/Users/vivek/OneDrive/Documents/Machine lear...,133,261,126,160
430,C:/Users/vivek/OneDrive/Documents/Machine lear...,98,297,102,191
431,C:/Users/vivek/OneDrive/Documents/Machine lear...,85,247,196,259


## Load images with corresponding annotation

In [5]:
filename = df['filepath'][0]
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('C:/Users/vivek/OneDrive/Documents/Machine learning/Detect-and-recognized-car-license/images',filename_image)
    return filepath_image
getFilename(filename)

image_path = list(df['filepath'].apply(getFilename))
image_path[:10]

['C:/Users/vivek/OneDrive/Documents/Machine learning/Detect-and-recognized-car-license/images\\Cars0.png',
 'C:/Users/vivek/OneDrive/Documents/Machine learning/Detect-and-recognized-car-license/images\\Cars1.png',
 'C:/Users/vivek/OneDrive/Documents/Machine learning/Detect-and-recognized-car-license/images\\Cars10.png',
 'C:/Users/vivek/OneDrive/Documents/Machine learning/Detect-and-recognized-car-license/images\\Cars100.png',
 'C:/Users/vivek/OneDrive/Documents/Machine learning/Detect-and-recognized-car-license/images\\Cars101.png',
 'C:/Users/vivek/OneDrive/Documents/Machine learning/Detect-and-recognized-car-license/images\\Cars102.png',
 'C:/Users/vivek/OneDrive/Documents/Machine learning/Detect-and-recognized-car-license/images\\Cars103.png',
 'C:/Users/vivek/OneDrive/Documents/Machine learning/Detect-and-recognized-car-license/images\\Cars104.png',
 'C:/Users/vivek/OneDrive/Documents/Machine learning/Detect-and-recognized-car-license/images\\Cars105.png',
 'C:/Users/vivek/OneDriv

In [6]:
def show_with_box(num):
    file_path = image_path[num]
    img = cv2.imread(file_path)  
    img = io.imread(file_path)
    fig = px.imshow(img)
    ymin = df.iloc[num]["ymin"]
    xmin = df.iloc[num]["xmin"]
    ymax = df.iloc[num]["ymax"]
    xmax = df.iloc[num]["xmax"]
    fig.update_layout(width=600, height=500, margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Figure With Bounding Box')
    fig.add_shape(type='rect',x0=xmin, x1=xmax, y0=ymin, y1=ymax, xref='x', yref='y',line_color='cyan')
    return fig

In [7]:
show_with_box(178)

## Data Processing

### Read Data and Normalize

In [8]:
data = []
output = []
for i in range(len(image_path)):
    image = image_path[i]
    img = cv2.imread(image)
    h,w,d = img.shape
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    load_image_array = load_image_arr/255.0 #normalization
    xmin,xmax,ymin,ymax = list(df.iloc[i:i+1,1:].values)[0]
    xmin,xmax = int(xmin)/w,int(xmax)/w
    ymin,ymax = int(ymin)/h,int(ymax)/h
    output.append((xmin,xmax,ymin,ymax))
    data.append(load_image_array)

### Spilt into train and test

In [9]:
X = np.array(data,dtype=np.float32)
Y = np.array(output,dtype=np.float32)

In [10]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,train_size=0.8,random_state=0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((346, 224, 224, 3), (87, 224, 224, 3), (346, 4), (87, 4))

## Deep learning for object detection

### Inception Resnet V2

In [11]:
ResnetV2 = InceptionResNetV2(include_top=False,
    weights="imagenet",
    input_tensor=Input(shape=(224,224,3)),
    pooling=None,
    classes=1000,
    classifier_activation="softmax")
finalOut = ResnetV2.output
finalOut = Flatten()(finalOut)
finalOut = Dense(500,activation='relu')(finalOut)
finalOut = Dense(250,activation='relu')(finalOut)
finalOut = Dense(4,activation='sigmoid')(finalOut)

model = Model(inputs=ResnetV2.input,outputs=finalOut)

In [12]:
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))

In [13]:
#model.summary()
#it gives long answer, So I don't write it

In [22]:
%reload_ext tensorboard

In [15]:
import datetime

In [16]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [17]:
print(tf.test.is_gpu_available('GPU'))
print(tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices('GPU'))

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True
True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [18]:
#with tf.device('GPU'):
#    tfb = TensorBoard('my_model1')
#    r = model.fit(
#           x=x_train,
#           y=y_train,
#           epochs = 150,
#           batch_size = 2,
#           validation_data = (x_test,y_test),
#           callbacks=[tfb]
#    )
#I save the model, so don't need to train again

In [19]:
model.save('./my_model1.keras')

In [77]:
%tensorboard --logdir="my_model4"

Reusing TensorBoard on port 6006 (pid 26424), started 23:54:31 ago. (Use '!kill 26424' to kill it.)

In [24]:
model = tf.keras.models.load_model('./my_model1.keras')

## Prediction

In [70]:
image = load_img(image_path[234])
image = np.array(image,dtype=np.uint8)
img1 = load_img(image_path[234],target_size=(224,224))
img1_array = img_to_array(img1)/255.0

h,w,d = image.shape
print('Height of the image =',h)
print('Width of the image =',w)
print(image_path[234])

Height of the image = 225
Width of the image = 400
C:/Users/vivek/OneDrive/Documents/Machine learning/Detect-and-recognized-car-license/images\Cars309.png


In [64]:
fig = px.imshow(image)
fig.update_layout(width=700, height=500,  margin=dict(l=10, r=10, b=10, t=10))

In [65]:
img1_array.shape

(224, 224, 3)

In [66]:
test_arr = img1_array.reshape(1,224,224,3)
test_arr.shape

(1, 224, 224, 3)

In [67]:
output = model.predict(test_arr)
output

1/1 [==============================] - 0s 76ms/step


array([[0.69959986, 0.24836814, 0.63705426, 0.95276487]], dtype=float32)

In [68]:
output_denorm = (output*np.array([w, w, h, h])).astype(np.int32)
output_denorm

array([[279,  99, 143, 214]])

In [69]:
xmin,xmax,ymin,ymax = output_denorm[0]
pt1 = (xmin,ymin)
pt2 = (xmax,ymax)
cv2.rectangle(image,pt1,pt2,(0,255,0),3)
fig = px.imshow(image)
fig.update_layout(width=700, height=500, margin=dict(l=10, r=10, b=10, t=10))

### Pipeline

In [33]:
def predictions(ind):
    im = load_img(image_path[ind])
    im = np.array(im,dtype=np.uint8)
    img2 = load_img(image_path[ind],target_size=(224,224))
    img2 = img_to_array(img2)/255.0
    h,w,d = im.shape
    test_arr = img2.reshape(1,224,224,3)
    
    output = model.predict(test_arr)
    output_denorm = (output*np.array([w,w,h,h])).astype(np.int32)
    xmin,xmax,ymin,ymax = output_denorm[0]
    pt1 = (xmin,ymin)
    pt2 = (xmax,ymax)
    cv2.rectangle(im,pt1,pt2,(0,255,0),3)
    
    return im, (xmin,xmax,ymin,ymax)

In [34]:
#for i in range(len(image_path)):
#    img4, cods = predictions(i)
#    
#    fig = px.imshow(img4)
#    fig.update_layout(width=700, height=500, margin=dict(l=10, r=10, b=10, t=10))
#    fig.show()
#    
#    xmin ,xmax,ymin,ymax = cods
#    roi = img4[ymin:ymax,xmin:xmax]
#    fig2 = px.imshow(roi)
#    fig2.update_layout(width=350, height=250, margin=dict(l=10, r=10, b=10, t=10))
#    fig2.show()
#    text = pt.image_to_string(roi)
#    print(text)

## Prediction with YOLO

In [69]:
# parsing
def parsing(path):
    parser = xet.parse(path).getroot()
    name = parser.find('filename').text
    filename = f'C:/Users/vivek/OneDrive/Documents/Machine learning/Detect-and-recognized-car-license/images/{name}'

    # width and height
    parser_size = parser.find('size')
    width = int(parser_size.find('width').text)
    height = int(parser_size.find('height').text)
    
    return filename, width, height
df[['filename','width','height']] = df['filepath'].apply(parsing).apply(pd.Series)
df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height,center_x,center_y,bb_width,bb_height
0,C:/Users/vivek/OneDrive/Documents/Machine lear...,226,419,125,173,C:/Users/vivek/OneDrive/Documents/Machine lear...,500,268,0.64500,0.555970,0.3860,0.179104
1,C:/Users/vivek/OneDrive/Documents/Machine lear...,134,262,128,160,C:/Users/vivek/OneDrive/Documents/Machine lear...,400,248,0.49500,0.580645,0.3200,0.129032
2,C:/Users/vivek/OneDrive/Documents/Machine lear...,140,303,5,148,C:/Users/vivek/OneDrive/Documents/Machine lear...,400,225,0.55375,0.340000,0.4075,0.635556
3,C:/Users/vivek/OneDrive/Documents/Machine lear...,175,214,114,131,C:/Users/vivek/OneDrive/Documents/Machine lear...,400,267,0.48625,0.458801,0.0975,0.063670
4,C:/Users/vivek/OneDrive/Documents/Machine lear...,167,240,202,220,C:/Users/vivek/OneDrive/Documents/Machine lear...,400,300,0.50875,0.703333,0.1825,0.060000


In [70]:
df['xmax'] = df.xmax.astype(int)
df['xmin'] = df.xmin.astype(int)
df['ymax'] = df.ymax.astype(int)
df['ymin'] = df.ymin.astype(int)
df.dtypes

filepath      object
xmin           int32
xmax           int32
ymin           int32
ymax           int32
filename      object
width          int64
height         int64
center_x     float64
center_y     float64
bb_width     float64
bb_height    float64
dtype: object

In [71]:
df['center_x'] = (df['xmax'] + df['xmin'])/(2*df['width'])
df['center_y'] = (df['ymax'] + df['ymin'])/(2*df['height'])

df['bb_width'] = (df['xmax'] - df['xmin'])/df['width']
df['bb_height'] = (df['ymax'] - df['ymin'])/df['height']
df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height,center_x,center_y,bb_width,bb_height
0,C:/Users/vivek/OneDrive/Documents/Machine lear...,226,419,125,173,C:/Users/vivek/OneDrive/Documents/Machine lear...,500,268,0.64500,0.555970,0.3860,0.179104
1,C:/Users/vivek/OneDrive/Documents/Machine lear...,134,262,128,160,C:/Users/vivek/OneDrive/Documents/Machine lear...,400,248,0.49500,0.580645,0.3200,0.129032
2,C:/Users/vivek/OneDrive/Documents/Machine lear...,140,303,5,148,C:/Users/vivek/OneDrive/Documents/Machine lear...,400,225,0.55375,0.340000,0.4075,0.635556
3,C:/Users/vivek/OneDrive/Documents/Machine lear...,175,214,114,131,C:/Users/vivek/OneDrive/Documents/Machine lear...,400,267,0.48625,0.458801,0.0975,0.063670
4,C:/Users/vivek/OneDrive/Documents/Machine lear...,167,240,202,220,C:/Users/vivek/OneDrive/Documents/Machine lear...,400,300,0.50875,0.703333,0.1825,0.060000


In [72]:
#!git clone https://github.com/ultralytics/yolov5

In [73]:
#!pip install -r ./yolov5/requirements.txt

In [74]:
#path = "C:/Users/vivek/OneDrive/Documents/Machine learning/Detect-and-recognized-car-license/yolov5/data_images"
#os.mkdir(path)

In [75]:
#path2 = "C:/Users/vivek/OneDrive/Documents/Machine learning/Detect-and-recognized-car-license/yolov5/data_images/test"
#os.mkdir(path2)

In [76]:
#path3 = "C:/Users/vivek/OneDrive/Documents/Machine learning/Detect-and-recognized-car-license/yolov5/data_images/train"
#os.mkdir(path3)

In [77]:
df_train = df.iloc[:200]
df_test = df.iloc[200:]

In [78]:
train_folder = './yolov5/data_images/train'

values = df_train[['filename','center_x','center_y','bb_width','bb_height']].values
for fname, x,y, w, h in values:
    image_name = os.path.split(fname)[-1]
    txt_name = os.path.splitext(image_name)[0]
    
    dst_image_path = os.path.join(train_folder,image_name)
    dst_label_file = os.path.join(train_folder,txt_name+'.txt')
    
    # copy each image into the folder
    copy(fname,dst_image_path)

    # generate .txt which has label info
    label_txt = f'0 {x} {y} {w} {h}'
    with open(dst_label_file,mode='w') as f:
        f.write(label_txt)
        
        f.close()

test_folder = './yolov5/data_images/test'

values = df_test[['filename','center_x','center_y','bb_width','bb_height']].values
for fname, x,y, w, h in values:
    image_name = os.path.split(fname)[-1]
    txt_name = os.path.splitext(image_name)[0]
    
    dst_image_path = os.path.join(test_folder,image_name)
    dst_label_file = os.path.join(test_folder,txt_name+'.txt')
    
    # copy each image into the folder
    copy(fname,dst_image_path)

    # generate .txt which has label info
    label_txt = f'0 {x} {y} {w} {h}'
    with open(dst_label_file,mode='w') as f:
        f.write(label_txt)
        
        f.close()

In [79]:
#!pip install GPUtil
#!pip install numba

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()  

c:\Users\vivek\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 62% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  8% |  2% |


In [80]:
#!python ./yolov5/train.py --data ./data.yaml --cfg ./yolov5/models/yolov5s.yaml --batch-size 8 --name Model --epochs 100

In [81]:
#!python yolov5/export.py --weights yolov5/runs/train/Model/weights/best.pt --img 640 640 --include onnx --optimize --simplify --opset 12

In [82]:
#!python ./yolov5/export.py --weight ./yolov5/runs/train/Model/weights/best.pt --include torchscript onnx

#### Test YOLO

In [85]:
INPUT_WIDTH =  640
INPUT_HEIGHT = 640

In [86]:
img = io.imread('./TEST/test3.jpg')

fig = px.imshow(img)
fig.update_layout(width=700, height=400, margin=dict(l=10, r=10, b=10, t=10))
fig.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)
fig.show()

In [87]:
net = cv2.dnn.readNetFromONNX('./yolov5/runs/train/Model/weights/best.onnx')
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [88]:
def get_detections(img,net):
    image = img.copy()
    row, col, d = image.shape

    max_rc = max(row,col)
    input_image = np.zeros((max_rc,max_rc,3),dtype=np.uint8)
    input_image[0:row,0:col] = image

    # 2. GET PREDICTION FROM YOLO MODEL
    blob = cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WIDTH,INPUT_HEIGHT),swapRB=True,crop=False)
    net.setInput(blob)
    preds = net.forward()
    detections = preds[0]
    
    return input_image, detections

def non_maximum_supression(input_image,detections):
    
    # 3. FILTER DETECTIONS BASED ON CONFIDENCE AND PROBABILIY SCORE
    
    # center x, center y, w , h, conf, proba
    boxes = []
    confidences = []

    image_w, image_h = input_image.shape[:2]
    x_factor = image_w/INPUT_WIDTH
    y_factor = image_h/INPUT_HEIGHT

    for i in range(len(detections)):
        row = detections[i]
        confidence = row[4] # confidence of detecting license plate
        if confidence > 0.4:
            class_score = row[5] # probability score of license plate
            if class_score > 0.25:
                cx, cy , w, h = row[0:4]

                left = int((cx - 0.5*w)*x_factor)
                top = int((cy-0.5*h)*y_factor)
                width = int(w*x_factor)
                height = int(h*y_factor)
                box = np.array([left,top,width,height])

                confidences.append(confidence)
                boxes.append(box)

    # 4.1 CLEAN
    boxes_np = np.array(boxes).tolist()
    confidences_np = np.array(confidences).tolist()
    
    # 4.2 NMS
    index = cv2.dnn.NMSBoxes(boxes_np,confidences_np,0.25,0.45)
    
    return boxes_np, confidences_np, index

def drawings(image,boxes_np,confidences_np,index):
    # 5. Drawings
    for ind in index:
        x,y,w,h =  boxes_np[ind]
        bb_conf = confidences_np[ind]
        conf_text = 'plate: {:.0f}%'.format(bb_conf*100)
        license_text = extract_text(image,boxes_np[ind])


        cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,255),2)
        cv2.rectangle(image,(x,y-30),(x+w,y),(255,0,255),-1)
        cv2.rectangle(image,(x,y+h),(x+w,y+h+25),(0,0,0),-1)


        cv2.putText(image,conf_text,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,255,255),1)
        cv2.putText(image,license_text,(x,y+h+27),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,0),1)

    return image

In [89]:
def yolo_predictions(img,net):
    # step-1: detections
    input_image, detections = get_detections(img,net)
    # step-2: NMS
    boxes_np, confidences_np, index = non_maximum_supression(input_image, detections)
    # step-3: Drawings
    result_img = drawings(img,boxes_np,confidences_np,index)
    return result_img

In [90]:
def extract_text(image,bbox):
    x,y,w,h = bbox
    roi = image[y:y+h, x:x+w]
    
    if 0 in roi.shape:
        return 'no number'
    
    else:
        text = pt.image_to_string(roi)
        text = text.strip()
        
        return text

In [91]:
img = io.imread('./TEST/test3.jpg')
results = yolo_predictions(img,net)

In [92]:
fig = px.imshow(img)
fig.update_layout(width=700, height=400, margin=dict(l=10, r=10, b=10, t=10))
fig.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)
fig.show()